Create a grid that allows the user to input a start and end point, then draw barriers. If you would like to remove start, end, or barriers then right click. When grid is filled out to your desire press the space bar and watch the grid use the aster pathfinding algorithm to find the shortest path. This algorithm is also called the manhatten algorithm or taxi algorithm because it used the shortest L shaped path and does not allow the path to use diagnols.

In [7]:
import pygame
import math
from queue import PriorityQueue

#Creating Window
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("Manhatten Path Finding Algorithm")

#Creating color variables that will color the nodes
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165 ,0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)

#Keep track of node, neighbors, colors
class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col
    
    #If node is red we have already looked at the node for the path
    def is_closed(self):
        return self.color == RED
    
    #Node is in open set
    def is_open(self):
        return self.color == GREEN
    
    #Node is an obstacle that can not be crossed
    def is_barrier(self):
        return self.color == BLACK
    
    #starting node
    def is_start(self):
        return self.color == ORANGE
    
    #end node
    def is_end(self):
        return self.color == TURQUOISE
    
    #reset grid to all whit
    def reset(self):
        self.color = WHITE
    
    #all make's actually change the color of the node
    def make_start(self):
        self.color = ORANGE

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK

    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
    
    #determine which squares are neighbors, exclude barrieres (black squares)
    def update_neighbors(self, grid):
        self.neighbors = []
        #check can you move down, if yes then check if node is a barrier, if not append
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # below neighbor
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # above neighbor
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # right neighbor
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # left neighbor
            self.neighbors.append(grid[self.row][self.col - 1])
    
    #if we compare two node objects, other spot is always > other spot
    def __lt__(self, other):
        return False

#using manhatten distance we will find the shortest L distance from two points
#heuristic function
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

#draw path using shortest path
def reconstruct_path(came_from, current, draw):
    #uses the 
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

#implment aster/manhatten algorithm
def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    #dict that keeps track of path
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = h(start.get_pos(), end.get_pos())

    open_set_hash = {start}

    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True
        #consider neighbors of current node
        for neighbor in current.neighbors:
            #moving one node adds one value
            temp_g_score = g_score[current] + 1
            #if we find a better way to reach the neighbor than start looking at that path instead
            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                #add to hash if value is not already present
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()
        
        if current != start:
            current.make_closed()
    #if no path found 
    return False

#create a nested list that creates the grid
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, j, gap, rows)
            grid[i].append(spot)

    return grid

#create grid lines
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))

#continue to update the grid with current colors
#Think of this as FPS, if given 100 FPS the game will update 100 times a second to keep track of clicks
def draw(win, grid, rows, width):
    win.fill(WHITE)

    for row in grid:
        for spot in row:
            spot.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()

#get mouse position and detemine what node is clicked on
def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos

    row = y // gap
    col = x // gap

    return row, col


def main(win, width):
    ROWS = 50
    grid = make_grid(ROWS, width)

    start = None
    end = None

    run = True
    while run:
        draw(win, grid, ROWS, width)
        #check what each event is that has happened in the window (ex: clicks)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]: #track left clicks
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                if not start and spot != end:
                    start = spot
                    start.make_start()

                elif not end and spot != start:
                    end = spot
                    end.make_end()

                elif spot != end and spot != start:
                    spot.make_barrier()

            elif pygame.mouse.get_pressed()[2]: # track right clicks
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN: #if space bar is pressed
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)
                    #call algorithm when space is pressed
                    algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)
                
                if event.key == pygame.K_c: #if c is pressed clear screen
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)

    pygame.quit()

main(WIN, WIDTH)